# Lluis Badia - SQL Queries using Python

## Importing libraries:

In [1]:
# Importing the necessary libraries:

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass            # password hiding tool
from IPython.display import display

## Odd numbers question:

In [2]:
# Odd numbers:

for x in range(0, 201):
    if "5" not in str(x) and x % 2 == 1:
        print(x)

1
3
7
9
11
13
17
19
21
23
27
29
31
33
37
39
41
43
47
49
61
63
67
69
71
73
77
79
81
83
87
89
91
93
97
99
101
103
107
109
111
113
117
119
121
123
127
129
131
133
137
139
141
143
147
149
161
163
167
169
171
173
177
179
181
183
187
189
191
193
197
199


## Importing CSV files:

In [3]:
# Importing CSV files. Therefore, we can make queries through python:
facturation = pd.read_csv('facturation_test.csv')
algo = pd.read_csv('algo_test.csv', sep = ';')

## Renaming the client id columns from both files:

Therefore, is possible to do a merge (in Python, not in MySQL):

In [4]:
new_f = {'numero_client': 'client_id'}
facturation.rename(columns = new_f, inplace = True)

In [5]:
new_a = {'id': 'client_id'}
algo.rename(columns = new_a, inplace = True)

In [6]:
facturation.columns

Index(['id', 'numero_facture', 'pnr', 'client_id', 'pays', 'date_facturation',
       'activite', 'type', 'montant_vente', 'nom_voyageur', 'description',
       'description_2', 'trajet_origine_destination', 'pays_destination',
       'prestataire', 'date_depart', 'date_retour'],
      dtype='object')

In [7]:
algo.columns

Index(['client_id', 'pays', 'nom_groupe', 'nom_entite'], dtype='object')

In [8]:
# Merging both CSV files:

merged = facturation.merge(algo, on = 'client_id', how = 'inner')

In [9]:
merged.head()

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
0,13096322,F-6200062442,O3W46D,1122,FR,08/02/2018,4,F,225.77,Jean-Jacques|GOLDMAN,Location Voiture ENTERPRISE LYON,LYON,LYON / LYON,FRANCE,ENTERPRISE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
1,13096323,F-6200062442,O3W46D,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE LYON,NaN,LYON / LYON,FRANCE,FRAIS RESA VOITURE ONLINE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
2,13097373,F-6200062452,S8TFEH,1122,FR,08/02/2018,4,F,119.99,Johnny|HALLYDAY,Location Voiture ENTERPRISE BORDEAUX,BORDEAUX,BORDEAUX / BORDEAUX,FRANCE,ENTERPRISE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
3,13097374,F-6200062452,S8TFEH,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE BORDEAUX,NaN,BORDEAUX / BORDEAUX,FRANCE,FRAIS RESA VOITURE ONLINE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
4,13097375,F-6200062454,Q7626D,1122,FR,08/02/2018,4,F,109.62,Michel|SARDOU,Location Voiture ENTERPRISE PARIS,PARIS,PARIS / PARIS,FRANCE,ENTERPRISE,04/12/2018,04/12/2018,FR,MICROSOFT,MICROSOFT


## MySQL Connection and query testing:

In [10]:
mysql_pw = getpass.getpass()

········


In [11]:
connection_string = 'mysql+pymysql://lbadia:' + mysql_pw + '@127.0.0.1:3306/test_sql'
engine = create_engine(connection_string)

In [12]:
testing = pd.read_sql_query('Select * from facturation_test limit 1', engine)
testing

,id,numero_facture,pnr,numero_client,pays,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour
0,13096322,F-6200062442,O3W46D,1122,FR,2018-08-02,4,F,225.77,Jean-Jacques|GOLDMAN,Location Voiture ENTERPRISE LYON,LYON,LYON / LYON,FRANCE,ENTERPRISE,2018-04-03,2018-04-04


In [13]:
testing2 = pd.read_sql_query('Select * from algo_test limit 1', engine)
testing2

,id,pays,nom_groupe,nom_entite
0,1122,FR,MICROSOFT,MICROSOFT


## Questions:

### Client:

Which SQL request would you do in order to obtain the air expenses in € per Customer group month by months (from invoice date) on the entire data set?

In [14]:
# SQL query:
client = pd.read_sql_query("select a.nom_groupe, round(sum(f.montant_vente) * 0.90909, 2) as air_expenses, date_format(f.date_facturation, '%%M') as month from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 5 group by nom_groupe, date_facturation", engine)
client

,nom_groupe,air_expenses,month
0,AFFLELOU,6362.27,August
1,TOYSRUS,6362.27,August
2,NIKE,631.54,August
3,AFFLELOU,3078.77,August
4,TOYSRUS,3078.77,August
...,...,...,...
114,MICROSOFT,153.00,August
115,MICROSOFT,105.40,August
116,MICROSOFT,502.66,August
117,MICROSOFT,145.97,August


In [22]:
# To get the air expenses in EUR grouped by 'nom_groupe':
merged_eur = merged
merged_eur['montant_vente'] = 0.90909 * merged_eur['montant_vente']
group_by = merged_eur[merged_eur['activite'] == 5].groupby('nom_groupe').apply(display)
group_by

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1947,13110117,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,464.322644,Jacques|DUTRONC,SHENYANG / NANKING,Economy,SHENYANG / NANKING,CHINE,CHINA EASTERN AIRLINES,08/21/2018,08/21/2018,FR,AFFLELOU,AFFLELOU
1949,13110118,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,3695.456869,Jacques|DUTRONC,GENEVE / FRANKFURT / SHENYANG - SHANGHAI / FRA...,Economy,GENEVE / SHENYANG,CHINE,LUFTHANSA,08/18/2018,08/24/2018,FR,AFFLELOU,AFFLELOU
1959,13114444,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,135.484355,Dave,MANCHESTER / AMSTERDAM / HANOVER,Economy,MANCHESTER / HANOVER,ALLEMAGNE,KLM,09/10/2018,09/10/2018,FR,AFFLELOU,AFFLELOU
1961,13114445,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,83.296179,Dave,HANOVER / MANCHESTER,Economy,HANOVER / MANCHESTER,ALLEMAGNE,FLYBE,09/11/2018,09/11/2018,FR,AFFLELOU,AFFLELOU
1969,13115110,F-2210004562,KFPYRI,231,CH,08/03/2018,5,F,185.716639,Marc|LAVOINE,ZURICH / LONDRES AR,Economy,ZURICH / LONDRES,ROYAUME-UNI,BRITISH AIRWAYS,08/25/2018,09/03/2018,FR,AFFLELOU,AFFLELOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,13538925,F-2210010546,S6LTRQ,231,FR,09/27/2018,5,F,107.667219,Aretha|FRANKLIN,BEIJING / SHENYANG,Economy,BEIJING / SHENYANG,CHINE,AIR CHINA,10/22/2018,10/22/2018,FR,AFFLELOU,AFFLELOU
3597,13540151,F-2210010552,NaN,231,CH,09/27/2018,5,F,1408.801346,Brett|HOFFMAN,Zurich Los Angeles,Economy,GENEVE / PHOENIX,ETATS-UNIS,LUFTHANSA,11/26/2018,11/26/2018,FR,AFFLELOU,AFFLELOU
3601,13550065,F-2210010554,LCB6Y2,231,FR,09/28/2018,5,F,425.794668,Billie|HOLIDAY,ZURICH / BEIJING - HONG KONG / ZURICH,Economy,ZURICH / BEIJING,CHINE,SWISS,11/11/2018,11/21/2018,FR,AFFLELOU,AFFLELOU
3609,13550514,F-2210010545,LGQ64O,231,CH,09/28/2018,5,F,146.567745,Jon|BUNCH,LONDRES / ZURICH,Economy,LONDRES / ZURICH,SUISSE,SWISS,10/19/2018,10/19/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
184,13310276,F-1140125211,ODEJI3,1122,FR,09/07/2018,5,F,129.740862,Jim|NABORS,LYON / BORDEAUX,Economy,LYON / BORDEAUX,FRANCE,AIR FRANCE,09/11/2018,09/11/2018,FR,MICROSOFT,MICROSOFT
327,13343242,F-6200050091,KEOQFN,1122,FR,09/11/2018,5,F,199.687299,Dennis|EDWARDS,MARSEILLE / BORDEAUX,Economy,MARSEILLE / BORDEAUX,FRANCE,AIR FRANCE,09/14/2018,09/14/2018,FR,MICROSOFT,MICROSOFT
338,13350615,F-6200050241,JXLTJO,1122,FR,09/12/2018,5,F,146.915460,Little|JIMMY|DICKENS,PARIS / SEVILLA AR,Economy,PARIS / SEVILLA,ESPAGNE,AIR FRANCE,09/24/2018,09/28/2018,FR,MICROSOFT,MICROSOFT
340,13350617,F-6200050242,JY6L5R,1122,FR,09/12/2018,5,F,268.435364,Etta|JAMES,PARIS / SEVILLA AR,First,PARIS / SEVILLA,ESPAGNE,AIR FRANCE,09/24/2018,09/28/2018,FR,MICROSOFT,MICROSOFT
347,13354246,F-6200050296,MNX6DA,1122,FR,09/12/2018,5,F,124.872863,Lil|PEEP,PARIS / BARCELONA AR,Economy,PARIS / BARCELONA,ESPAGNE,IBERIA,09/17/2018,09/19/2018,FR,MICROSOFT,MICROSOFT
440,13397344,F-6200051525,PNTAVE,1122,FR,09/15/2018,5,F,239.003841,Patricia|MORISON,PAU / PARIS AR,Economy,PAU / PARIS,FRANCE,AIR FRANCE,09/26/2018,09/26/2018,FR,MICROSOFT,MICROSOFT
499,13411574,F-6200051494,LWGA3U,1122,FR,09/17/2018,5,F,67.630412,Lou|REED,PAU / PARIS AR,Economy,PAU / PARIS,FRANCE,AIR FRANCE,10/23/2018,10/24/2018,FR,MICROSOFT,MICROSOFT
564,13441743,F-6200052456,QJQ2WU,1122,FR,09/20/2018,5,F,60.042792,Etta|JAMES,PARIS / SEVILLA,Economy,PARIS / SEVILLA,ESPAGNE,AIR FRANCE,10/02/2018,10/02/2018,FR,MICROSOFT,MICROSOFT
566,13441753,F-6200052460,QK254Y,1122,FR,09/20/2018,5,F,60.042792,Little|JIMMY|DICKENS,PARIS / SEVILLA,Economy,PARIS / SEVILLA,ESPAGNE,AIR FRANCE,10/02/2018,10/02/2018,FR,MICROSOFT,MICROSOFT
571,13443298,F-6200052922,KL7LNS,1122,FR,09/20/2018,5,F,260.959509,Dennis|EDWARDS,BORDEAUX / LYON / METZ,Economy,BORDEAUX / METZ,FRANCE,AIR FRANCE,09/24/2018,09/24/2018,FR,MICROSOFT,MICROSOFT


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1512,13112495,F-1140124220,TAWJDJ,7026,FR,08/01/2018,5,F,214.738357,Alain|BASHUNG,PARIS / MUNICH,Economy,PARIS / MUNICH,ALLEMAGNE,AIR FRANCE,08/09/2018,08/09/2018,FR,NIKE,CONVERSE
1522,13125891,F-9240000041,K7H5VG,7026,FR,08/03/2018,5,F,127.331699,Yannick|NOAH,PARIS / ZURICH AR,Economy,PARIS / ZURICH,SUISSE,AIR FRANCE,09/28/2018,09/28/2018,FR,NIKE,CONVERSE
1536,13126925,F-9240000055,TRDHBI,7026,FR,08/01/2018,5,F,216.607321,Pierre|BACHELET,PARIS / BREMEN AR,First,PARIS / BREMEN,ALLEMAGNE,AIR FRANCE,09/10/2018,09/13/2018,FR,NIKE,CONVERSE
1554,13126946,F-9240000094,K92LGM,7026,FR,08/03/2018,5,F,218.091315,Julien|DORÃ‰,RENNES / PARIS / AMSTERDAM / RENNES,Economy,RENNES / AMSTERDAM,PAYS-BAS,AIR FRANCE,09/12/2018,09/14/2018,FR,NIKE,CONVERSE
1557,13139283,F-9240000222,LGO23A,7026,FR,08/09/2018,5,F,373.774140,Eddy|MITCHELL,PARIS / HAMBURG AR,Economy,PARIS / HAMBURG,ALLEMAGNE,AIR FRANCE,08/21/2018,08/23/2018,FR,NIKE,CONVERSE
1566,13141734,F-9240000241,NCYISS,7026,FR,08/10/2018,5,F,250.292134,Adamo,BORDEAUX / AJACCIO AR,First,BORDEAUX / AJACCIO,FRANCE,AIR FRANCE,09/11/2018,09/13/2018,FR,NIKE,CONVERSE
1568,13141736,F-9240000242,NCY2QV,7026,FR,08/10/2018,5,F,101.271761,Michel|DELPECH,BORDEAUX / AJACCIO AR,Economy,BORDEAUX / AJACCIO,FRANCE,VOLOTEA,09/11/2018,09/13/2018,FR,NIKE,CONVERSE
1570,13141738,F-9240000242,NAE42H,7026,FR,08/10/2018,5,F,101.271761,Michel|FUGAIN,BORDEAUX / AJACCIO AR,Economy,BORDEAUX / AJACCIO,FRANCE,VOLOTEA,09/11/2018,09/13/2018,FR,NIKE,CONVERSE
1572,13141740,F-9240000244,NCYFMP,7026,FR,08/10/2018,5,F,101.271761,Gilbert|MONTAGNÃ‰,BORDEAUX / AJACCIO AR,Economy,BORDEAUX / AJACCIO,FRANCE,VOLOTEA,09/11/2018,09/13/2018,FR,NIKE,CONVERSE
1575,13142404,F-9240000290,N9NG3W,7026,FR,08/10/2018,5,F,74.826855,Pierre|PERRET,NANTES / AJACCIO AR,Economy,NANTES / AJACCIO,FRANCE,VOLOTEA,09/11/2018,09/13/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1946,13110117,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,464.322644,Jacques|DUTRONC,SHENYANG / NANKING,Economy,SHENYANG / NANKING,CHINE,CHINA EASTERN AIRLINES,08/21/2018,08/21/2018,CH,TOYSRUS,TOYSRUS
1948,13110118,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,3695.456869,Jacques|DUTRONC,GENEVE / FRANKFURT / SHENYANG - SHANGHAI / FRA...,Economy,GENEVE / SHENYANG,CHINE,LUFTHANSA,08/18/2018,08/24/2018,CH,TOYSRUS,TOYSRUS
1958,13114444,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,135.484355,Dave,MANCHESTER / AMSTERDAM / HANOVER,Economy,MANCHESTER / HANOVER,ALLEMAGNE,KLM,09/10/2018,09/10/2018,CH,TOYSRUS,TOYSRUS
1960,13114445,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,83.296179,Dave,HANOVER / MANCHESTER,Economy,HANOVER / MANCHESTER,ALLEMAGNE,FLYBE,09/11/2018,09/11/2018,CH,TOYSRUS,TOYSRUS
1968,13115110,F-2210004562,KFPYRI,231,CH,08/03/2018,5,F,185.716639,Marc|LAVOINE,ZURICH / LONDRES AR,Economy,ZURICH / LONDRES,ROYAUME-UNI,BRITISH AIRWAYS,08/25/2018,09/03/2018,CH,TOYSRUS,TOYSRUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,13538925,F-2210010546,S6LTRQ,231,FR,09/27/2018,5,F,107.667219,Aretha|FRANKLIN,BEIJING / SHENYANG,Economy,BEIJING / SHENYANG,CHINE,AIR CHINA,10/22/2018,10/22/2018,CH,TOYSRUS,TOYSRUS
3596,13540151,F-2210010552,NaN,231,CH,09/27/2018,5,F,1408.801346,Brett|HOFFMAN,Zurich Los Angeles,Economy,GENEVE / PHOENIX,ETATS-UNIS,LUFTHANSA,11/26/2018,11/26/2018,CH,TOYSRUS,TOYSRUS
3600,13550065,F-2210010554,LCB6Y2,231,FR,09/28/2018,5,F,425.794668,Billie|HOLIDAY,ZURICH / BEIJING - HONG KONG / ZURICH,Economy,ZURICH / BEIJING,CHINE,SWISS,11/11/2018,11/21/2018,CH,TOYSRUS,TOYSRUS
3608,13550514,F-2210010545,LGQ64O,231,CH,09/28/2018,5,F,146.567745,Jon|BUNCH,LONDRES / ZURICH,Economy,LONDRES / ZURICH,SUISSE,SWISS,10/19/2018,10/19/2018,CH,TOYSRUS,TOYSRUS


""


### Datas Understanding:

Delete all customer payment and charge lines.

In [23]:
delete = merged.drop(merged[merged.activite == 11].index)
delete2 = merged.drop(merged[merged.activite == 6].index)

### Destination Country:

Get the air expenses by destination country and trip in euros for all entities on all period AUGUST, SEPTEMBER 2018.

In [28]:
# SQL query:
destination = pd.read_sql_query("select a.nom_entite, round(sum(f.montant_vente) * 0.90909, 2) as air_expenses, f.pays_destination from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 5 and f.date_facturation between '2018-08-01' and '2018-09-30' group by a.nom_entite, f.pays_destination", engine)
destination

,nom_entite,air_expenses,pays_destination
0,AFFLELOU,31030.79,CHINE
1,TOYSRUS,31030.79,CHINE
2,CONVERSE,2513.77,ALLEMAGNE
3,AFFLELOU,7407.77,ALLEMAGNE
4,TOYSRUS,7407.77,ALLEMAGNE
...,...,...,...
56,NIKE,107.85,BOLIVIE
57,AFFLELOU,600.27,"RUSSIE, FEDERATION DE"
58,TOYSRUS,600.27,"RUSSIE, FEDERATION DE"
59,MICROSOFT,120.85,INDE


In [58]:
# Using python functions:

group_by2 = merged_eur[merged_eur['activite'] == 5].groupby(['nom_entite', 'pays_destination']).apply(display)
group_by2

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
2151,13154743,F-2210004450,JTFJ3S,231,FR,08/15/2018,5,F,3876.206162,Georges|BRASSENS,ZURICH / JOHANNESBURG AR,Economy,ZURICH / JOHANNESBURG,AFRIQUE DU SUD,SWISS,10/06/2018,10/11/2018,FR,AFFLELOU,AFFLELOU
2189,13166877,F-2210009029,RRFH48,231,CH,08/20/2018,5,F,2625.521641,Barbara|COOK,ZURICH / LONDRES / JOHANNESBURG / LONDRES / ZU...,Economy,ZURICH / JOHANNESBURG,AFRIQUE DU SUD,BRITISH AIRWAYS,10/06/2018,10/13/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1959,13114444,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,135.484355,Dave,MANCHESTER / AMSTERDAM / HANOVER,Economy,MANCHESTER / HANOVER,ALLEMAGNE,KLM,09/10/2018,09/10/2018,FR,AFFLELOU,AFFLELOU
1961,13114445,F-2210004595,RQ4KTM,231,CH,08/06/2018,5,F,83.296179,Dave,HANOVER / MANCHESTER,Economy,HANOVER / MANCHESTER,ALLEMAGNE,FLYBE,09/11/2018,09/11/2018,FR,AFFLELOU,AFFLELOU
1999,13123864,F-2210004626,MYLWBW,231,FR,08/07/2018,5,F,121.079053,Nino|FERRER,Modification HANOVER / ZURICH,Economy,HANOVER / ZURICH,ALLEMAGNE,SWISS,08/08/2018,08/08/2018,FR,AFFLELOU,AFFLELOU
2117,13153709,F-2210004425,J3E6A3,231,CH,08/14/2018,5,F,166.406083,Nino|FERRER,ZURICH / HANOVER,Economy,ZURICH / HANOVER,ALLEMAGNE,SWISS,09/10/2018,09/10/2018,FR,AFFLELOU,AFFLELOU
2119,13153710,F-2210004425,J3E6A3,231,CH,08/14/2018,5,F,144.394532,Nino|FERRER,HANOVER / ZURICH,Economy,HANOVER / ZURICH,ALLEMAGNE,SWISS,09/11/2018,09/11/2018,FR,AFFLELOU,AFFLELOU
2183,13163408,F-2210004994,R256F2,231,FR,08/17/2018,5,F,337.158593,Bankroll|FRESH,HANOVER / ZURICH,Premium,HANOVER / ZURICH,ALLEMAGNE,SWISS,09/11/2018,09/11/2018,FR,AFFLELOU,AFFLELOU
2185,13163409,F-2210004994,R256F2,231,FR,08/17/2018,5,F,162.680573,Bankroll|FRESH,ZURICH / HANOVER,Economy,ZURICH / HANOVER,ALLEMAGNE,SWISS,09/10/2018,09/10/2018,FR,AFFLELOU,AFFLELOU
2287,13176017,F-2210004965,QTMVPY,231,CH,08/17/2018,5,F,210.460230,Nino|FERRER,ZURICH / HANOVER - DUSSELDORF / ZURICH,Economy,ZURICH / HANOVER,ALLEMAGNE,SWISS,09/17/2018,09/21/2018,FR,AFFLELOU,AFFLELOU
2473,13260981,F-2210004542,UUVU97,231,CH,08/13/2018,5,F,186.834292,Elliott|SMITH,ZURICH / DUSSELDORF AR,Economy,ZURICH / DUSSELDORF,ALLEMAGNE,SWISS,09/11/2018,09/13/2018,FR,AFFLELOU,AFFLELOU
2479,13260984,F-2210004426,JSS47Y,231,FR,08/14/2018,5,F,300.431280,Elliott|SMITH,ZURICH / FRANKFURT AR,Economy,ZURICH / FRANKFURT,ALLEMAGNE,LUFTHANSA,09/24/2018,09/28/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
2197,13169006,F-2210009059,KJNV22,231,CH,08/20/2018,5,F,291.334828,Jean|FERRAT,ZURICH / VIENNE AR,Economy,ZURICH / VIENNE,AUTRICHE,AUSTRIAN,08/21/2018,08/22/2018,FR,AFFLELOU,AFFLELOU
3583,13536583,F-2210010520,UGVUNY,231,FR,09/27/2018,5,F,193.943806,James|COTTON,ZURICH / VIENNE - BUDAPEST / ZURICH,Economy,ZURICH / VIENNE,AUTRICHE,SWISS,10/23/2018,10/25/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
2557,13274634,F-2210004952,QREVGJ,231,FR,08/17/2018,5,F,134.863437,Jaco|PASTORIUS,SAULT STE MARIE / TORONTO,Economy,SAULT STE MARIE / TORONTO,CANADA,AIR CANADA,08/31/2018,08/31/2018,FR,AFFLELOU,AFFLELOU
2571,13274641,F-2210009009,OX6A3J,231,CH,08/17/2018,5,F,453.798080,Jaco|PASTORIUS,AGUASCALIENTES / HOUSTON / TORONTO / SAULT STE...,Economy,AGUASCALIENTES / SAULT STE MARIE,CANADA,AIR CANADA,08/29/2018,08/29/2018,FR,AFFLELOU,AFFLELOU
2685,13274701,F-2210009615,SGTD5M,231,CH,09/03/2018,5,F,154.018764,Jackie|WILSON,BOSTON / TORONTO / CALGARY,Economy,BOSTON / CALGARY,CANADA,AIR CANADA,10/10/2018,10/11/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1947,13110117,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,464.322644,Jacques|DUTRONC,SHENYANG / NANKING,Economy,SHENYANG / NANKING,CHINE,CHINA EASTERN AIRLINES,08/21/2018,08/21/2018,FR,AFFLELOU,AFFLELOU
1949,13110118,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,3695.456869,Jacques|DUTRONC,GENEVE / FRANKFURT / SHENYANG - SHANGHAI / FRA...,Economy,GENEVE / SHENYANG,CHINE,LUFTHANSA,08/18/2018,08/24/2018,FR,AFFLELOU,AFFLELOU
2207,13169354,F-2210009064,KDSAVF,231,FR,08/20/2018,5,F,3345.942004,Barry|WHITE,ZURICH / COPENHAGEN / SHANGHAI / COPENHAGEN / ...,Economy,ZURICH / SHANGHAI,CHINE,SCANDINAVIAN AIRLINE,09/02/2018,09/07/2018,FR,AFFLELOU,AFFLELOU
2935,13276246,F-2210009515,IUTANS,231,CH,09/05/2018,5,F,2227.171558,Guru,MILAN / SHANGHAI / BEIJING / MILAN,Economy,MILAN / SHANGHAI,CHINE,AIR CHINA,09/21/2018,09/30/2018,FR,AFFLELOU,AFFLELOU
3105,13328552,F-2210009445,LJUWRA,231,CH,09/10/2018,5,F,525.172629,Aretha|FRANKLIN,ZURICH / HELSINKI-VANTAA / BEIJING / HELSINKI-...,Economy,ZURICH / BEIJING,CHINE,FINNAIR,10/21/2018,10/27/2018,FR,AFFLELOU,AFFLELOU
3135,13349848,F-2210009922,KNHTXG,231,CH,09/11/2018,5,F,5207.920511,Mac|MILLER,ZURICH / SINGAPORE / SHANGHAI / SINGAPORE / ZU...,Economy,ZURICH / SHANGHAI,CHINE,SINGAPORE AIRLINES,09/22/2018,09/28/2018,FR,AFFLELOU,AFFLELOU
3187,13363199,F-2210010001,IUTANS,231,CH,09/12/2018,5,F,23.594892,Guru,Modification MILAN / SHANGHAI / BEIJING / MILAN,Economy,MILAN / SHANGHAI,CHINE,AIR CHINA,09/22/2018,09/30/2018,FR,AFFLELOU,AFFLELOU
3377,13448113,F-2210010405,SMA64G,231,CH,09/20/2018,5,F,4842.354910,Jenni|RIVERA,FRANKFURT / SHANGHAI - BEIJING / FRANKFURT,Economy,FRANKFURT / SHANGHAI,CHINE,LUFTHANSA,10/15/2018,10/19/2018,FR,AFFLELOU,AFFLELOU
3561,13512036,F-2210010669,S69X9V,231,CH,09/26/2018,5,F,162.928941,Aretha|FRANKLIN,SHENYANG / HANGZHOU,Economy,SHENYANG / HANGZHOU,CHINE,CHINA SOUTHERN AIRLINES,10/23/2018,10/23/2018,FR,AFFLELOU,AFFLELOU
3567,13512039,F-2210010650,S6X6PN,231,FR,09/26/2018,5,F,166.033532,Aretha|FRANKLIN,CHENGDU / BEIJING,Economy,CHENGDU / BEIJING,CHINE,CHINA SOUTHERN AIRLINES,10/26/2018,10/26/2018,FR,AFFLELOU,AFFLELOU


...

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1982,13119943,F-2210004609,RGX658,231,CH,08/07/2018,5,F,242.127059,Georges|BRASSENS,ZURICH / STOCKHOLM AR,Economy,ZURICH / STOCKHOLM,SUEDE,SCANDINAVIAN AIRLINE,08/20/2018,08/23/2018,CH,TOYSRUS,TOYSRUS
2006,13124792,F-2210004641,UK5LKY,231,CH,08/08/2018,5,F,318.872551,Francis|LALANNE,FRANKFURT / STOCKHOLM AR,Economy,FRANKFURT / STOCKHOLM,SUEDE,SCANDINAVIAN AIRLINE,08/14/2018,08/16/2018,CH,TOYSRUS,TOYSRUS
2028,13130121,F-2210004654,UG3EYM,231,FR,08/08/2018,5,F,187.113705,M.|POKORA,ZURICH / STOCKHOLM AR,Eco,ZURICH / STOCKHOLM,SUEDE,SWISS,09/14/2018,09/18/2018,CH,TOYSRUS,TOYSRUS
2054,13139753,F-2210004512,KT3TXA,231,FR,08/10/2018,5,F,2215.156790,Georges|BRASSENS,ZURICH / COPENHAGEN / CHICAGO / STOCKHOLM / ZU...,Economy,ZURICH / CHICAGO,SUEDE,SCANDINAVIAN AIRLINE,09/12/2018,09/20/2018,CH,TOYSRUS,TOYSRUS
2066,13144903,F-2210004552,N5AA7N,231,FR,08/10/2018,5,F,476.554733,Georges|MOUSTAKI,ZURICH / STOCKHOLM / KALMAR - COPENHAGEN / ZURICH,Economy,ZURICH / KALMAR,SUEDE,SWISS,09/17/2018,09/19/2018,CH,TOYSRUS,TOYSRUS
2076,13146884,F-2210004542,UIYUB3,231,FR,08/13/2018,5,F,360.101521,Georges|BRASSENS,ZURICH / STOCKHOLM AR,Eco,ZURICH / STOCKHOLM,SUEDE,SWISS,09/03/2018,09/04/2018,CH,TOYSRUS,TOYSRUS
2080,13148854,F-2210004591,K7A5DK,231,CH,08/13/2018,5,F,124.432011,Bernard|LAVILLIERS,ZURICH / STOCKHOLM AR,Economy,ZURICH / STOCKHOLM,SUEDE,SWISS,08/27/2018,08/29/2018,CH,TOYSRUS,TOYSRUS
2100,13149281,F-2210004402,UY3525,231,CH,08/13/2018,5,F,360.101521,Bernard|LAVILLIERS,ZURICH / STOCKHOLM AR,Economy,ZURICH / STOCKHOLM,SUEDE,SWISS,09/03/2018,09/04/2018,CH,TOYSRUS,TOYSRUS
2124,13153713,F-2210004424,JSV529,231,FR,08/14/2018,5,F,152.652744,Adam|YAUCH,FRANKFURT / STOCKHOLM,Economy,FRANKFURT / STOCKHOLM,SUEDE,LUFTHANSA,09/03/2018,09/03/2018,CH,TOYSRUS,TOYSRUS
2126,13153714,F-2210004424,JSV529,231,FR,08/14/2018,5,F,198.880105,Adam|YAUCH,STOCKHOLM / FRANKFURT,Economy,STOCKHOLM / FRANKFURT,SUEDE,LUFTHANSA,09/06/2018,09/06/2018,CH,TOYSRUS,TOYSRUS


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1988,13122499,F-2210004620,TW7HM3,231,FR,08/07/2018,5,F,254.141827,Jean|FERRAT,BARCELONA / MUNICH / HANOVER / FRANKFURT / ZURICH,Economy,BARCELONA / ZURICH,SUISSE,LUFTHANSA,08/12/2018,08/13/2018,CH,TOYSRUS,TOYSRUS
2760,13275427,F-2210004455,MC75DX,231,CH,08/15/2018,5,F,62.495419,Jani|LANE,HELSINKI-VANTAA / STOCKHOLM / ZURICH,Economy,HELSINKI-VANTAA / ZURICH,SUISSE,SCANDINAVIAN AIRLINE,09/28/2018,09/28/2018,CH,TOYSRUS,TOYSRUS
3484,13477870,F-2210010520,P625VL,231,FR,09/24/2018,5,F,281.306999,Aretha|FRANKLIN,VENISE / MUNICH / LONDRES / ZURICH,Economy,VENISE / ZURICH,SUISSE,LUFTHANSA,10/17/2018,10/19/2018,CH,TOYSRUS,TOYSRUS
3494,13486833,F-2210010554,OR3APQ,231,FR,09/24/2018,5,F,119.495711,Claude|NOUGARO,VENISE / ZURICH,Economy,VENISE / ZURICH,SUISSE,SWISS,10/10/2018,10/10/2018,CH,TOYSRUS,TOYSRUS
3608,13550514,F-2210010545,LGQ64O,231,CH,09/28/2018,5,F,146.567745,Jon|BUNCH,LONDRES / ZURICH,Economy,LONDRES / ZURICH,SUISSE,SWISS,10/19/2018,10/19/2018,CH,TOYSRUS,TOYSRUS


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
2022,13127129,F-2210004602,SORAJX,231,FR,08/06/2018,5,F,1884.05215,Marc|LAVOINE,BANGKOK / DUBAI / ZURICH,Economy,BANGKOK / ZURICH,THAILANDE,EMIRATES,08/18/2018,08/20/2018,CH,TOYSRUS,TOYSRUS


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
3180,13357977,F-2210009964,MK399A,231,FR,09/12/2018,5,F,4062.947317,Jeff|BUCKLEY,ZURICH / WASHINGTON / NEW ORLEANS / WASHINGTON...,Economy,ZURICH / TUNIS,TUNISIE,UNITED AIRLINES,09/29/2018,10/10/2018,CH,TOYSRUS,TOYSRUS


""


### Hotel:

Get the Hotel TOP 10 by expenditure in euros from AUGUST 15th 2018, for all customers.

In [68]:
# SQL query:
hotel = pd.read_sql_query("select a.id, f.prestataire, round(f.montant_vente * 0.90909, 2) as hotel_exp from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 2 and f.date_facturation = '2018-08-15' order by f.montant_vente desc limit 10", engine)
hotel

,id,prestataire,hotel_exp
0,231,SCANDIC HOTELS,104.79
1,231,SCANDIC HOTELS,104.79
2,231,RADISSON HOTELS,91.32
3,231,RADISSON HOTELS,91.32
4,231,MARRIOTT HTLS RSTS,59.35
5,231,MARRIOTT HTLS RSTS,59.35


In [73]:
py_hotel_date = merged_eur.query('date_facturation == "08/15/2018"')
py_hotel = py_hotel_date[py_hotel_date['activite'] == 2].sort_values(by = 'montant_vente', ascending = False)
py_hotel.head(10) # limit 10

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
2768,13275431,F-2210004455,MBRB9Y,231,CH,08/15/2018,2,F,71.573243,Jani|LANE,Reservation SEURAHUONE HELSINKI,HELSINKI-VANTAA,HELSINKI-VANTAA / HELSINKI-VANTAA,FINLANDE,SCANDIC HOTELS,09/27/2018,09/28/2018,CH,TOYSRUS,TOYSRUS
2769,13275431,F-2210004455,MBRB9Y,231,CH,08/15/2018,2,F,71.573243,Jani|LANE,Reservation SEURAHUONE HELSINKI,HELSINKI-VANTAA,HELSINKI-VANTAA / HELSINKI-VANTAA,FINLANDE,SCANDIC HOTELS,09/27/2018,09/28/2018,FR,AFFLELOU,AFFLELOU
2732,13275413,F-2210004452,MBR24Q,231,FR,08/15/2018,2,F,62.371235,Jani|LANE,Reservation PARK INN BY RADISSON STOCKHOLM SOLNA,Stockholm,Stockholm / Stockholm,SUEDE,RADISSON HOTELS,09/26/2018,09/27/2018,CH,TOYSRUS,TOYSRUS
2733,13275413,F-2210004452,MBR24Q,231,FR,08/15/2018,2,F,62.371235,Jani|LANE,Reservation PARK INN BY RADISSON STOCKHOLM SOLNA,Stockholm,Stockholm / Stockholm,SUEDE,RADISSON HOTELS,09/26/2018,09/27/2018,FR,AFFLELOU,AFFLELOU
2140,13154624,F-2210004444,VJ8UXW,231,FR,08/15/2018,2,F,40.539750,Alan|VEGA,Reservation COURTYARD BEAVERCREEK MARRIOTT,DAYTON,DAYTON / DAYTON,ETATS-UNIS,MARRIOTT HTLS RSTS,07/29/2018,07/31/2018,CH,TOYSRUS,TOYSRUS
2141,13154624,F-2210004444,VJ8UXW,231,FR,08/15/2018,2,F,40.539750,Alan|VEGA,Reservation COURTYARD BEAVERCREEK MARRIOTT,DAYTON,DAYTON / DAYTON,ETATS-UNIS,MARRIOTT HTLS RSTS,07/29/2018,07/31/2018,FR,AFFLELOU,AFFLELOU


### Top 5 of Nike group:

Create a TOP 5 of travelers for the client NIKE in descending order of expenditure over the month of SEPTEMBER 2018.
Add NAME | FIRST NAME of traveler, Number of trips (beware of credit notes) and air expenses.

In [81]:
# SQL query:
nike = pd.read_sql_query("select substring_index(f.nom_voyageur, '|', 1) as name, substring_index(f.nom_voyageur, '|', -1) as surname, count(f.description) as num_trips, f.montant_vente from facturation_test f inner join algo_test a on f.numero_client = a.id where a.nom_groupe = 'NIKE' and f.date_facturation between '2018-09-01' and '2018-09-30'group by f.nom_voyageur order by f.montant_vente desc limit 5", engine)
nike

,name,surname,num_trips,montant_vente
0,Buddy,MILES,1,889.73
1,Joe,SOUTH,1,686.30
2,BRUEL,PATRICK,2,632.37
3,Blowfly,Blowfly,2,601.45
4,Savannah,CHURCHILL,2,577.50


In [97]:
py_nike_date = merged_eur.query('date_facturation >= "09/01/2018" and date_facturation <= "09/30/2018"')
py_nike = py_nike_date.query('nom_groupe == "NIKE"').groupby('nom_voyageur').apply(display)
py_nike2 = py_nike.groupby(['nom_voyageur'])['montant_vente'].sum()
py_nike2

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1774,13266939,F-9240001255,NaN,7026,FR,09/04/2018,11,F,-240.202213,ATC,.,NaN,NaN,ALLEMAGNE,ATC,09/04/2018,0000-00-00,FR,NIKE,CONVERSE
1783,13275688,F-9240001222,NaN,7026,FR,09/05/2018,11,F,-92.131845,ATC,.,NaN,NaN,FRANCE,ATC,09/05/2018,0000-00-00,FR,NIKE,CONVERSE
1787,13283401,F-9240001245,NaN,7026,FR,09/06/2018,11,F,-300.431280,ATC,.,NaN,NaN,ALLEMAGNE,ATC,09/06/2018,0000-00-00,FR,NIKE,CONVERSE
1809,13348657,A-9240000122,NaN,7026,FR,09/11/2018,11,A,240.202213,ATC,.,NaN,NaN,ALLEMAGNE,ATC,09/11/2018,0000-00-00,FR,NIKE,CONVERSE
1818,13348826,F-9240001569,NaN,7026,FR,09/11/2018,11,F,-240.202213,ATC,.,NaN,NaN,ALLEMAGNE,ATC,09/11/2018,0000-00-00,FR,NIKE,CONVERSE
1822,13348830,F-9240001554,NaN,7026,FR,09/11/2018,11,F,-267.684053,ATC,.,NaN,NaN,DANEMARK,ATC,09/11/2018,0000-00-00,FR,NIKE,CONVERSE
1909,13491539,F-9240002511,NaN,7026,FR,09/25/2018,11,F,-49.040121,ATC,.,NaN,NaN,SUISSE,ATC,09/25/2018,0000-00-00,FR,NIKE,CONVERSE
1917,13501429,F-9240002556,NaN,7026,FR,09/25/2018,11,F,-91.753085,ATC,.,NaN,NaN,ROYAUME-UNI,ATC,09/25/2018,0000-00-00,FR,NIKE,CONVERSE
1934,13535632,F-9240002520,NaN,7026,FR,09/27/2018,11,F,-267.895165,ATC,.,NaN,NaN,ROYAUME-UNI,ATC,09/27/2018,0000-00-00,FR,NIKE,CONVERSE
1938,13537722,F-9240002525,NaN,7026,FR,09/27/2018,11,F,-176.868555,ATC,.,NaN,NaN,ALLEMAGNE,ATC,09/27/2018,0000-00-00,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1843,13388227,A-9240000152,UGJTQA,7026,FR,09/14/2018,1,A,-2.483673,Alain|BASHUNG,BRUXELLES (BE) / PARIS (FR),NaN,BRUXELLES (BE) / PARIS (FR),BELGIQUE,SNCF,08/31/2018,08/31/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1763,13258730,F-9240001221,QY37QG,7026,FR,09/04/2018,4,F,392.650054,BRUEL|PATRICK,Location Voiture AVIS STAVANGER,Sola (NO),Sola (NO) / Sola (NO),NORVEGE,AVIS,08/28/2018,08/31/2018,FR,NIKE,CONVERSE
1935,13537719,F-9240002525,IWWTBW,7026,FR,09/27/2018,5,F,176.868555,BRUEL|PATRICK,PARIS / BREMEN AR,Economy,PARIS / BREMEN,ALLEMAGNE,AIR FRANCE,11/12/2018,11/16/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
3642,13319565,F-9240001454,L92G8M,7024,FR,09/10/2018,4,F,116.130334,Betty|HUTTON,Location Voiture AVIS PARIS,ORLY (FR),ORLY (FR) / PARIS,FRANCE,AVIS,09/05/2018,09/06/2018,FR,NIKE,NIKE
3652,13342635,F-9240001551,KER3AF,7024,FR,09/11/2018,1,F,46.568866,Betty|HUTTON,LYON (FR) / PARIS (FR),NaN,LYON (FR) / PARIS (FR),FRANCE,SNCF,09/13/2018,09/13/2018,FR,NIKE,NIKE
3670,13386762,F-9240001561,PDRSDA,7024,FR,09/13/2018,1,F,60.229067,Betty|HUTTON,PARIS (FR) / LYON (FR),NaN,PARIS (FR) / LYON (FR),FRANCE,SNCF,09/13/2018,09/13/2018,FR,NIKE,NIKE
3684,13429819,F-9240002120,NLJ46G,7024,FR,09/19/2018,5,F,68.301004,Betty|HUTTON,Modification PARIS / MARSEILLE,Economy,PARIS / MARSEILLE,FRANCE,AIR FRANCE,09/19/2018,09/19/2018,FR,NIKE,NIKE
3685,13429820,F-9240002120,NLJ46G,7024,FR,09/19/2018,5,F,31.045911,Betty|HUTTON,SIEGE SUPPLEMENTAIRE,Economy,PARIS / MARSEILLE,FRANCE,AIR FRANCE,09/19/2018,09/19/2018,FR,NIKE,NIKE
3695,13538253,F-9240002525,J4JFCD,7024,FR,09/27/2018,5,F,249.248991,Betty|HUTTON,MARSEILLE / PARIS / MARSEILLE,Economy,MARSEILLE / PARIS,FRANCE,AIR FRANCE,10/02/2018,10/03/2018,FR,NIKE,NIKE
3697,13548136,F-9240002554,L64R5Y,7024,FR,09/28/2018,2,F,95.323365,Betty|HUTTON,Reservation HOLIDAY INN PARIS CDG AIRPORT,PARIS,PARIS / PARIS,FRANCE,HOLIDAY INN,10/02/2018,10/03/2018,FR,NIKE,NIKE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1828,13360884,F-9240001645,VJSKVH,7026,FR,09/12/2018,2,F,44.693693,Billy|PAUL,Reservation INTERCONTINENTAL WARSZAWA,VARSOVIE,VARSOVIE / VARSOVIE,POLOGNE,InterContinental Warszawa,09/18/2018,09/20/2018,FR,NIKE,CONVERSE


...

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1911,13494908,F-9240002522,RCWHYG,7026,FR,09/25/2018,1,F,60.229067,Tammy|GRIMES,LYON (FR) / PARIS (FR),NaN,LYON (FR) / PARIS (FR),FRANCE,SNCF,10/01/2018,10/01/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1914,13495602,F-9240002525,RE3DYA,7026,FR,09/25/2018,1,F,58.987231,Teena|MARIE,STRASBOURG (FR) / MASSY (FR),NaN,STRASBOURG (FR) / MASSY (FR),FRANCE,SNCF,10/02/2018,10/02/2018,FR,NIKE,CONVERSE
1918,13503491,F-9240002552,RE3DYA,7026,FR,09/25/2018,2,F,39.384843,Teena|MARIE,Reservation NOVOTEL ORLEANS CHARBONNIERE,ORLEANS,ORLEANS / ORLEANS,FRANCE,ACCOR HOTELS,10/02/2018,10/05/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1931,13535628,F-9240002520,NaN,7026,FR,09/27/2018,2,F,115.068564,Troy|GENTRY,Location,BIRMINGHAM,/ BIRMINGHAM,ROYAUME-UNI,HILTON PARIS CDG,08/28/2018,08/29/2018,FR,NIKE,CONVERSE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1779,13274110,F-9240001212,OJDQN2,7026,FR,09/05/2018,2,F,33.150824,Yannick|NOAH,Reservation PENTAHOTEL AIRPORT BRUSSELS,BRUXELLES,BRUXELLES / BRUXELLES,BELGIQUE,PENTAHOTEL AIRPORT BRUSSELS,09/11/2018,09/13/2018,FR,NIKE,CONVERSE


KeyError: 'nom_voyageur'

### Potential Saving:

In [106]:
# SQL query:
saving = pd.read_sql_query("select case when f.trajet_origine_destination = 'ZURICH / STOCKHOLM' then round(sum(f.montant_vente * 3), 2) else round(sum(f.montant_vente), 2) end as expenses from facturation_test f inner join algo_test a on f.numero_client = a.id where a.nom_groupe = 'AFFLELOU'", engine)
saving

,expenses
0,251934.51


In [111]:
py_destination = merged.query('nom_groupe == "AFFLELOU" and trajet_origine_destination == "ZURICH / STOCKHOLM"')
py_destination_sum = py_destination['montant_vente'].sum()
round(py_destination_sum, 2)

4906.25